In [ ]:
suppressMessages({
    library(dplyr)
    library(data.table)
    library(ggplot2)
    library(ggrepel)
    library(UpSetR)
    library(pheatmap)
    library(grid)
    library(RColorBrewer)
    library(viridis)
    library(UpSetR)
})

In [ ]:
root <- ### FILL THIS IN

source(paste0(root,"crispr_utils.R"))
source(paste0(root,"chirp_utils.R"))

In [ ]:
crispr_root <- paste0(root,"crispr/")
crispr_outputs <- paste0(crispr_root,"outputs/")
if (!dir.exists(crispr_outputs)) dir.create(crispr_outputs)

In [ ]:
e_counts <- read.delim(paste0(crispr_root,"exp_counts_table.tsv"),stringsAsFactors=F)
m_counts <- read.delim(paste0(crispr_root,"mito_counts_table.tsv"),stringsAsFactors=F)

In [ ]:
sig_t <- 0.001

e_all <- normalize_counts(e_counts)
e_merged <- guide_zscores(create_condition_list(e_all,merge_reps = T))
e_nomerge <- guide_zscores(create_condition_list(e_all,merge_reps = F))
e_genes <- gene_zscores(e_merged)

ez_genes <- create_zscore_table(e_genes,mb="Gene")

m_all <- normalize_counts(m_counts)
m_merged <- guide_zscores(create_condition_list(m_all,merge_reps = T))
m_nomerge <- guide_zscores(create_condition_list(m_all,merge_reps = F))
m_genes <- gene_zscores(m_merged)

mz_genes <- create_zscore_table(m_genes,mb="Gene")

e_reps_gene_zscore <- create_zscore_table(gene_zscores(e_nomerge),mb = "Gene")
m_reps_gene_zscore <- create_zscore_table(gene_zscores(m_nomerge),mb = "Gene")

In [ ]:
ez <- e_reps_gene_zscore[,which(endsWith(colnames(e_reps_gene_zscore),"_z"))]
mz <- m_reps_gene_zscore[,which(endsWith(colnames(m_reps_gene_zscore),"_z"))]

pca_plt(ez,nm="exp-pool-reps",group1="virus",offset=4,corrout = crispr_outputs)
pca_plt(mz,nm="mito-pool-reps",group1="virus",offset=4,corrout = crispr_outputs)

In [ ]:
proviral <- list()
antiviral <- list()

for (v in names(e_genes)) {
    e_genes[[v]] <- e_genes[[v]] %>% arrange(z)
    antiviral[[v]] <- e_genes[[v]]$Gene[which(e_genes[[v]]$z < 0 & e_genes[[v]]$fdr <= sig_t)]
    e_genes[[v]] <- e_genes[[v]] %>% arrange(desc(z))
    proviral[[v]] <- e_genes[[v]]$Gene[which(e_genes[[v]]$z > 0 & e_genes[[v]]$fdr <= sig_t)] 
}

for (v in names(m_genes)) {
    m_genes[[v]] <- m_genes[[v]] %>% arrange(z)
    antiviral[[v]] <- m_genes[[v]]$Gene[which(m_genes[[v]]$z < 0 & m_genes[[v]]$fdr <= sig_t)]
    m_genes[[v]] <- m_genes[[v]] %>% arrange(desc(z))
    proviral[[v]] <- m_genes[[v]]$Gene[which(m_genes[[v]]$z > 0 & m_genes[[v]]$fdr <= sig_t)] 
}


In [ ]:
pv <- "exp_sars2"

results <- e_genes

for (virus in names(results)) {
    plt <- crispr_volcano(results[[virus]],virus,c1=antiviral[[pv]],c2=proviral[[pv]],sig_t=sig_t)
    print(plt)
    ggsave(paste0(crispr_outputs,virus,"-sars2-colored-001.eps"),plt)
    plt <- crispr_volcano(results[[virus]],virus,sig_t=sig_t)
    print(plt)
    ggsave(paste0(crispr_outputs,virus,"-001.eps"),plt)
}

In [ ]:
pv <- "mito_sars2"

results <- m_genes

for (virus in names(results)) {
    plt <- crispr_volcano(results[[virus]],virus,c1=antiviral[[pv]],c2=proviral[[pv]],sig_t=sig_t)
    print(plt)
    ggsave(paste0(crispr_outputs,virus,"-sars2-colored-001.eps"),plt)
    plt <- crispr_volcano(results[[virus]],virus,sig_t=sig_t)
    print(plt)
    ggsave(paste0(crispr_outputs,virus,"-001.eps"),plt)
}

In [ ]:
chirp_root <- paste0(root,"chirp/")
chirp_outputs <- paste0(chirp_root,"outputs/")
if (!dir.exists(chirp_outputs)) dir.create(chirp_outputs)

In [ ]:
huh_data_raw <- read.table(paste0(chirp_root,"200814_ChIRP-All_Log2_Imput.txt"),
                       sep="\t",header = TRUE,stringsAsFactors=FALSE)
huh_data_raw$uniprot <- unlist(lapply(huh_data_raw$Majority.protein.IDs,parse_uniprot))
huh_data_raw$species <- unlist(lapply(huh_data_raw$Majority.protein.IDs,parse_species))

vero_data_raw <- read.table(paste0(chirp_root,"Vero_Log2_Imput.txt"),
                       sep="\t",header = TRUE,stringsAsFactors=FALSE)
vero_data_raw$uniprot <- unlist(lapply(vero_data_raw$Majority.protein.IDs,parse_uniprot))
vero_data_raw$species <- unlist(lapply(vero_data_raw$Majority.protein.IDs,parse_species))

In [ ]:
covd1 <- c('LFQ.intensity.CoV.D1.1','LFQ.intensity.CoV.D1.2','LFQ.intensity.CoV.D1.3')
covd2 <- c('LFQ.intensity.CoV.D2.1','LFQ.intensity.CoV.D2.2','LFQ.intensity.CoV.D2.3')
covm  <- c('LFQ.intensity.Mock1','LFQ.intensity.Mock2','LFQ.intensity.Mock3')

zika <- c('LFQ.intensity.ZV1','LFQ.intensity.ZV2','LFQ.intensity.ZV3')
deng <- c('LFQ.intensity.DV1','LFQ.intensity.DV2','LFQ.intensity.DV3')
flaviC <- c('LFQ.intensity.FlaviC1','LFQ.intensity.FlaviC2','LFQ.intensity.FlaviC3')

huh_data <- data.frame(
    "huhcov.d1.mean"=apply(huh_data_raw[,covd1],1,mean),
    "huhcov.d2.mean"=apply(huh_data_raw[,covd2],1,mean),
    "huhcov.mock.mean"=apply(huh_data_raw[,covm],1,mean),
    "zika.mean"=apply(huh_data_raw[,zika],1,mean),
    "deng.mean"=apply(huh_data_raw[,deng],1,mean),
    "flavic.mean"=apply(huh_data_raw[,flaviC],1,mean),
    "rv"=huh_data_raw[,'LFQ.intensity.RV'],
    "rvc"=huh_data_raw[,'LFQ.intensity.RV_C'],
    "majority.protein"=huh_data_raw[,"Majority.protein.IDs"],
    "uniprot"=huh_data_raw[,"uniprot"],
    "species"=huh_data_raw[,"species"],
    stringsAsFactors=FALSE
)

huh_data[,rfmt(covd1)] <- huh_data_raw[,covd1]
huh_data[,rfmt(covd2)] <- huh_data_raw[,covd2]
huh_data[,rfmt(covm)] <- huh_data_raw[,covm]
huh_data[,rfmt(zika)] <- huh_data_raw[,zika]
huh_data[,rfmt(deng)] <- huh_data_raw[,deng]
huh_data[,rfmt(flaviC)] <- huh_data_raw[,flaviC]

cmock <- c("Mock1","Mock2","Mock3")
huh_data[,"huhcov.d1.enrich.1"] <- huh_data[,"CoV.D1.1"] - apply(huh_data[,cmock],1,mean)
huh_data[,"huhcov.d1.enrich.2"] <- huh_data[,"CoV.D1.2"] - apply(huh_data[,cmock],1,mean)
huh_data[,"huhcov.d1.enrich.3"] <- huh_data[,"CoV.D1.3"] - apply(huh_data[,cmock],1,mean)
huh_data[,"huhcov.d2.enrich.1"] <- huh_data[,"CoV.D2.1"] - apply(huh_data[,cmock],1,mean)
huh_data[,"huhcov.d2.enrich.2"] <- huh_data[,"CoV.D2.2"] - apply(huh_data[,cmock],1,mean)
huh_data[,"huhcov.d2.enrich.3"] <- huh_data[,"CoV.D2.3"] - apply(huh_data[,cmock],1,mean)

fmock <- c("FlaviC1","FlaviC2","FlaviC3")
huh_data[,"zika.enrich.1"] <- huh_data[,"ZV1"] - apply(huh_data[,fmock],1,mean)
huh_data[,"zika.enrich.2"] <- huh_data[,"ZV2"] - apply(huh_data[,fmock],1,mean)
huh_data[,"zika.enrich.3"] <- huh_data[,"ZV3"] - apply(huh_data[,fmock],1,mean)
huh_data[,"deng.enrich.1"] <- huh_data[,"DV1"] - apply(huh_data[,fmock],1,mean)
huh_data[,"deng.enrich.2"] <- huh_data[,"DV2"] - apply(huh_data[,fmock],1,mean)
huh_data[,"deng.enrich.3"] <- huh_data[,"DV3"] - apply(huh_data[,fmock],1,mean)

huh_data[,"rv.enrich.1"] <- huh_data[,"rv"] - huh_data[,"rvc"]

huh_data[,"huhcov.d2.enrich.mean"] <- apply(huh_data[,c("huhcov.d2.enrich.1",
                                                        "huhcov.d2.enrich.2","huhcov.d2.enrich.3")],1,mean)
huh_data[,"huhcov.d1.enrich.mean"] <- apply(huh_data[,c("huhcov.d1.enrich.1",
                                                        "huhcov.d1.enrich.2","huhcov.d1.enrich.3")],1,mean)

huh_data[,"zika.enrich.mean"] <- apply(huh_data[,c("zika.enrich.1","zika.enrich.2","zika.enrich.3")],1,mean)
huh_data[,"deng.enrich.mean"] <- apply(huh_data[,c("deng.enrich.1","deng.enrich.2","deng.enrich.3")],1,mean)

In [ ]:
covd1 <- c('LFQ.intensity.V.Day1.1','LFQ.intensity.V.Day1.2','LFQ.intensity.V.Day1.3')
covd2 <- c('LFQ.intensity.V.Day2.1','LFQ.intensity.V.Day2.2','LFQ.intensity.V.Day2.3')
covm  <- c('LFQ.intensity.V.Mock1','LFQ.intensity.V.Mock2','LFQ.intensity.V.Mock3')

vero_data <- data.frame(
    "verocov.d1.mean"=apply(vero_data_raw[,covd1],1,mean),
    "verocov.d2.mean"=apply(vero_data_raw[,covd2],1,mean),
    "verocov.mock.mean"=apply(vero_data_raw[,covm],1,mean),
    "uniprot"=vero_data_raw[,"uniprot"],
    "majority.protein"=vero_data_raw[,"Majority.protein.IDs"],
    "species"=vero_data_raw[,"species"],
    stringsAsFactors=FALSE
)

vero_data[,rfmt(covd1)] <- vero_data_raw[,covd1]
vero_data[,rfmt(covd2)] <- vero_data_raw[,covd2]
vero_data[,rfmt(covm)] <- vero_data_raw[,covm]

vmock <- c("V.Mock1","V.Mock2","V.Mock3")
vero_data[,"vero.d1.enrich.1"] <- vero_data[,"V.Day1.1"] - apply(vero_data[,vmock],1,mean)
vero_data[,"vero.d1.enrich.2"] <- vero_data[,"V.Day1.2"] - apply(vero_data[,vmock],1,mean)
vero_data[,"vero.d1.enrich.3"] <- vero_data[,"V.Day1.3"] - apply(vero_data[,vmock],1,mean)
vero_data[,"vero.d2.enrich.1"] <- vero_data[,"V.Day2.1"] - apply(vero_data[,vmock],1,mean)
vero_data[,"vero.d2.enrich.2"] <- vero_data[,"V.Day2.2"] - apply(vero_data[,vmock],1,mean)
vero_data[,"vero.d2.enrich.3"] <- vero_data[,"V.Day2.3"] - apply(vero_data[,vmock],1,mean)

vero_data[,"vero.d2.enrich.mean"] <- apply(vero_data[,c("vero.d2.enrich.1",
                                                        "vero.d2.enrich.2","vero.d2.enrich.3")],1,mean)
vero_data[,"vero.d1.enrich.mean"] <- apply(vero_data[,c("vero.d1.enrich.1",
                                                        "vero.d1.enrich.2","vero.d1.enrich.3")],1,mean)

In [ ]:

gene_uniprot_lookup <- read.delim(paste0(chirp_root,"uniprot_human_lookup.tsv"),stringsAsFactors=F)
vero_lookup <- read.delim(paste0(chirp_root,"uniprot_monkey_lookup.tsv"),stringsAsFactors=F)

huh_data$gene <- unlist(lapply(huh_data$uniprot,function(x){
    if (!endsWith(x,"HUMAN") & !endsWith(x,"CHLSB")) return(x)
    lk <- gene_uniprot_lookup[
        which(gene_uniprot_lookup$Entry.name == x)[1],
        "Gene.names"
    ]
    if (lk == "") return(x)
    return(lk)
}))
huh_data$gene1 <- unlist(lapply(huh_data$gene,function(x){
    strsplit(x," ")[[1]][1]
}))
huh_data$name <- make.unique(huh_data$gene1)
vero_data$gene <- unlist(lapply(vero_data$uniprot,function(x){
    if (!endsWith(x,"HUMAN") & !endsWith(x,"CHLSB")) return(x)
    lk <- vero_lookup[
        which(vero_lookup$Entry.name == x)[1],
        "Gene.names"
    ]
    if (lk == "") return(x)
    return(lk)
}))
vero_data$gene1 <- unlist(lapply(vero_data$gene,function(x){
    strsplit(x," ")[[1]][1]
}))
vero_data$name <- make.unique(vero_data$gene1)

In [ ]:
sel_cols <- function(df) {
    sel <- colnames(df)
    sel <- sel[grep("enrich",sel)]
    return(sel[which(!endsWith(sel,"mean"))])
}

pca_plt(huh_data,sel_cols(huh_data),nm = "huh-chirp",offset=2,corrout = chirp_outputs)
pca_plt(vero_data,sel_cols(vero_data),nm = "vero-chirp",offset=2,corrout = chirp_outputs)

In [ ]:
everything <- merge(
    huh_data, vero_data,
    by.x = "name", by.y = "name", all.x=T, all.y=T
)

everything$huhd1_expanded <- everything$huhcov.d1.enrich.mean >= 1 & everything$species.x == "HUMAN"
everything$huhd2_expanded <- everything$huhcov.d2.enrich.mean >= 1 & everything$species.x == "HUMAN"
everything$verod1_expanded <- everything$vero.d1.enrich.mean >= 1 & everything$species.y == "CHLSB"
everything$verod2_expanded <- everything$vero.d2.enrich.mean >= 1 & everything$species.y == "CHLSB"

everything$huh_expanded <- everything$huhd1_expanded | everything$huhd2_expanded
everything$vero_expanded <- everything$verod1_expanded | everything$verod2_expanded
everything$sars_expanded <- everything$huh_expanded | everything$vero_expanded

everything$zika_expanded <- everything$zika.enrich.mean >= 1 & everything$species.x == "HUMAN"
everything$deng_expanded <- everything$deng.enrich.mean >= 1 & everything$species.x == "HUMAN"
everything$rv_expanded   <- everything$rv.enrich.1 >= 1 & everything$species.x == "HUMAN"

In [ ]:
plt <- upset(
    fromList(list(
        #huhd1 = everything[which(everything$huhd1_expanded == T),"name"],
        huhd2 = everything[which(everything$huhd2_expanded == T),"name"],
        zika = everything[which(everything$zika_expanded == T),"name"],
        deng = everything[which(everything$deng_expanded == T),"name"],
        rv = everything[which(everything$rv_expanded == T),"name"]
    )), sets=rev(c("huhd2","zika","deng","rv")),
    order.by = "freq",
    point.size = 3.5, line.size = 2,text.scale=2,
    keep.order = TRUE
) 

plt

In [ ]:

sarsfasta <- c(
    "SARS-leaderProtein",
    "SARS-nsp2",
    "SARS-nsp3",
    "SARS-nsp4",
    "SARS-3CLikeProteinase",
    "SARS-nsp6",
    "SARS-nsp7",
    "SARS-nsp8",
    "SARS-nsp9",
    "SARS-nsp10",
    "SARS-RdRp",
    "SARS-helicase",
    "SARS-35exonuclease",
    "SARS-endoRNAse",
    "SARS-2'O-riboseMethyltransferase",
    "SARS-SurfaceGlycoProtein",
    "SARS-ORRF3a",
    "SARS-Env",
    "SARS-MemGlycoProtein",
    "SARS-ORF6",
    "SARS-ORF7aYP009724395",
    "SARS-ORF7b",
    "SARS-ORF8",
    "SARS-NucleocapsidPhosPro",
    "SARS-ORF10"
)

make_bar_plot(everything,sw="viral_proteins",genelist = sarsfasta,fixord=T)
make_gene_set_heatmap(everything,sw="sars",colsel="allaverages",glist=sarsfasta,fixord=T,rmempty=F)